In [1]:
import csv
import datetime


In [2]:
# read course xt data

courseList = []
with open("./MOOC_data/xtData/xt_6month(course_talk)/CourseXTdata.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        courseList.append(row)
        
len(courseList)


29979

In [3]:
courseList[0]

{'course_id': 'courseTalk-1',
 'course_length': '8',
 'effort_hours (per_week)': '3',
 'enrollment_type': 'FREE audit/Pay for certificate',
 'level': 'Introductory',
 'price': '49',
 'provider': 'edX',
 'school': 'Massachusetts Institute of Technology',
 'subject': 'Engineering',
 'time': '1',
 'time_avg_helpful_review_score': '0',
 'time_avg_high_review': '0',
 'time_avg_low_review': '0',
 'time_avg_moderate_review': '0',
 'time_avg_review_length': '0',
 'time_review_count': '0',
 'time_review_rating': '0',
 'title': 'Introduction to Aerospace Engineering: Astronautics and Human Spaceflight',
 'total_review_count': '12',
 'total_review_rating': '5',
 'url': '36498'}

In [4]:
# read network xt data

networkList = []
with open("./MOOC_data/xtData/xt_6month(course_talk)/network_centrality_two_mode.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        networkList.append(row)
        
len(networkList)


1598

In [5]:
networkList[0]

{'': '1',
 'betweenness': '0',
 'closeness': '0.000997008973080758',
 'constraint': '0.285398576825755',
 'degree': '9',
 'eigenvector': '0.161201687738163',
 'time': '1',
 'transitivity': '1.00238095238095',
 'url': '34704'}

<h3> Add network centrality to the courseXT data</h3>

In [6]:
## set the network attribute to the data
for i, course in enumerate(courseList):
    
    for net in networkList:
        if course['url'] == net['url'] and course['time'] == net['time']:
            courseList[i]['degree'] = net['degree']
            courseList[i]['closeness'] = net['closeness']
            courseList[i]['betweenness'] = net['betweenness']
            courseList[i]['eigenvector'] = net['eigenvector']
            courseList[i]['transitivity'] = net['transitivity']
            courseList[i]['constraint'] = net['constraint']
            

In [7]:
## set NA to the data which don't have network attr
for i, course in enumerate(courseList):
    if 'degree' not in course.keys():
        courseList[i]['degree'] = "NA"
        courseList[i]['closeness'] = "NA"
        courseList[i]['betweenness'] = "NA"
        courseList[i]['eigenvector'] = "NA"
        courseList[i]['transitivity'] = "NA"
        courseList[i]['constraint'] = "NA"

<h3> Add Flag of session life to the courseXT data </h3>

In [8]:
# read session data
sessionList = []
with open("./MOOC_data/xtData/xt_1year(class_central)/ClassCentral_session.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        
        flag = False
        for index, session in enumerate(sessionList):
            if row['url'] == session['url']:
                flag = True
                sessionList[index]['sessions'].append(row['session'])
        
        if flag == False:

            sessionList.append(dict({
                'url': row['url'],
                'title': row['title'],
                'provider': row['provider'],
                'sessions': [row['session']]
            }))

      

In [9]:
# sorting each session
for index, session in enumerate(sessionList):
    sessionList[index]['sessions'].sort()


In [10]:
# add session start time stamp
#### T1: 13.1~6 / T2: 13.7~12 / T3; 14.1~6 / T4: 14.7~14.12 / T5: 15.1~6 / T6: 15.7~12 / T7: 16.1~16.6 / T8: 16.7~16.12 / T9: 17.1~6


for i, course in enumerate(courseList):
    for session in sessionList:
        if course['title'] == session['title'] and course['provider'] == session['provider']:
                   
            if session['sessions'][0] == "null":
                # session이 null일 경우 무시
                continue 
                    
            # session 첫번째가 2012년, 11년인 경우가 있으므로 처리해야함.
            date = datetime.datetime.strptime(session['sessions'][0], "%Y-%m-%d")
            
            if date.year == 2013:

                # T1:
                if date.month > 0 and date.month < 7:
                    courseList[i]['published_time'] = "1";
                    
                
                else:
                    courseList[i]['published_time'] = "2";
                   

            elif date.year == 2014:
                
                if date.month > 0 and date.month < 7:
                    courseList[i]['published_time'] = "3";

                
                else:
                    courseList[i]['published_time'] = "4";


            elif date.year == 2015:
                
                if date.month > 0 and date.month < 7:
                    courseList[i]['published_time'] = "5";

                # T3
                else:
                    courseList[i]['published_time'] = "6";


            elif date.year == 2016:
                
                if date.month > 0 and date.month < 7:
                    courseList[i]['published_time'] = "7";

                # T4
                else:
                    courseList[i]['published_time'] = "8";


            elif date.year == 2017:
                
                courseList[i]['published_time'] = "9";
                
            elif date.year == 2011 or date.year == 2012:
                courseList[i]['published_time'] = "0";
                
            elif date.year == 2018:
                courseList[i]['published_time'] = "NA";


In [11]:
def calculate_session(course, session):
    
    if course['course_length'] == "NA":
        week = 10
    else:
        week = round(float(course['course_length']))
        
    if course['time'] == '1':
        for d in session['sessions']:
            if d == "null":
                return 2
            date = datetime.datetime.strptime(d, "%Y-%m-%d")
            date_to = date + datetime.timedelta(days=week*7)
            if date.year == 2013 or date_to.year == 2013:
                if (date.month > 0 and date.month < 7) or (date_to.month > 0 and date_to.month < 7):
                    return 0
                            
        ## 아무것도 매칭안될 경우 return 1
        return 1
            
    elif course['time'] == '2':
        for d in session['sessions']:
            if d == "null":
                return 2
            date = datetime.datetime.strptime(d, "%Y-%m-%d")
            date_to = date + datetime.timedelta(days=week*7)
            if date.year == 2013 or date_to.year == 2013:
                if (date.month > 6 and date.month <= 12) or (date_to.month > 6 and date_to.month <= 12):
                    return 0
                            
        return 1
                
            
    elif course['time'] == '3':
        for d in session['sessions']:
            if d == "null":
                return 2
            date = datetime.datetime.strptime(d, "%Y-%m-%d")
            date_to = date + datetime.timedelta(days=week*7)
            if date.year == 2014 or date_to.year == 2014:
                if (date.month > 0 and date.month < 7) or (date_to.month > 0 and date_to.month < 7):
                    return 0
        return 1
    
    elif course['time'] == '4':
        for d in session['sessions']:
            if d == "null":
                return 2
            date = datetime.datetime.strptime(d, "%Y-%m-%d")
            date_to = date + datetime.timedelta(days=week*7)
            if date.year == 2014 and date_to.year == 2014:
                if (date.month > 6 and date.month <= 12) or (date_to.month > 6 and date_to.month <= 12):
                    return 0
               
        return 1
    
    elif course['time'] == '5':
        for d in session['sessions']:
            if d == "null":
                return 2
            date = datetime.datetime.strptime(d, "%Y-%m-%d")
            date_to = date + datetime.timedelta(days=week*7)
            if date.year == 2015 and date_to.year == 2015:
                if (date.month > 0 and date.month < 7) or (date_to.month > 0 and date_to.month < 7):
                    return 0
               
        return 1
    
    elif course['time'] == '6':
        for d in session['sessions']:
            if d == "null":
                return 2
            date = datetime.datetime.strptime(d, "%Y-%m-%d")
            date_to = date + datetime.timedelta(days=week*7)
            if date.year == 2015 and date_to.year == 2015:
                if (date.month > 6 and date.month <= 12) or (date_to.month > 6 and date_to.month <= 12):
                    return 0
               
        return 1
    
    elif course['time'] == '7':
        for d in session['sessions']:
            if d == "null":
                return 2
            date = datetime.datetime.strptime(d, "%Y-%m-%d")
            date_to = date + datetime.timedelta(days=week*7)
            if date.year == 2016 and date_to.year == 2016:
                if (date.month > 0 and date.month < 7) or (date_to.month > 0 and date_to.month < 7):
                    return 0
               
        return 1
    
    elif course['time'] == '8':
        for d in session['sessions']:
            if d == "null":
                return 2
            date = datetime.datetime.strptime(d, "%Y-%m-%d")
            date_to = date + datetime.timedelta(days=week*7)
            if date.year == 2016 and date_to.year == 2016:
                if (date.month > 6 and date.month <= 12) or (date_to.month > 6 and date_to.month <= 12):
                    return 0
               
        return 1
    
    elif course['time'] == '9':
        for d in session['sessions']:
            if d == "null":
                return 2
            date = datetime.datetime.strptime(d, "%Y-%m-%d")
            date_to = date + datetime.timedelta(days=week*7)
            if date.year == 2017 and date_to.year == 2017:
                if (date.month > 0 and date.month < 7) or (date_to.month > 0 and date_to.month < 7):
                    return 0
               
        return 1
    

In [12]:
# Session date를 기준으로 해당 course length(week) + 4 week(한 달)을 life time으로 설정. 
# course length가 NA인 경우.... 평균 6.21 week 이므로 6week + 4week = 10week

for i, course in enumerate(courseList):
    
    for session in sessionList:
        
        if course['title'] == session['title'] and course['provider'] == session['provider']:
            
            result = calculate_session(course, session)
            if result == 0:
                courseList[i]['session_open'] = "TRUE"
            elif result == 1:
                courseList[i]['session_open'] = "FALSE"
            else:
                courseList[i]['session_open'] = "NA"
            
            

In [13]:
## set "NA" if each dictionary dosen't have the keys
for index, course in enumerate(courseList):
    if 'session_open' not in course.keys():
        courseList[index]['session_open'] = "NA"
    if 'published_time' not in course.keys():
        courseList[index]['published_time'] = "NA"

<h3> Subject, provider, area mapping </h3>

In [14]:
subject_url_map = []
with open('./MOOC_Network_data/url_subject_info.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        subject_url_map.append(row);
        
subject_url_map[0]
        

{'area': 'Physical Science and Engineering',
 'provider': 'Coursera',
 'subject': 'Environmental Science and Sustainability',
 'url': 'https://www.class-central.com/mooc/1307/coursera-fundamentals-of-global-energy-business'}

In [15]:
for index, course in enumerate(courseList):
    
    flag = False
    for subject in subject_url_map:
        if course['url'] == subject['url']:
            flag = True
            courseList[index]['subject'] = subject['subject']
            courseList[index]['area'] = subject['area']
            courseList[index]['provider'] = subject['provider']
    
    if flag == False:
        courseList[index]['area'] = "NA"
        
        

<h3> Write file </h3>

In [16]:
with open('./MOOC_data/xtData/xt_6month(course_talk)/CourseXTdata(two_mode_network).csv', 'w', newline = '') as write_file:
    
    fieldnames = ['course_id', 'title', 'provider','area', 'subject', 'school', 'total_review_rating', 'total_review_count', 'published_time', 
                  'time', 'session_open', 'time_review_count', 'time_review_rating', 'time_avg_helpful_review_score', 'time_avg_moderate_review', 'time_avg_review_length',
                  'total_review_rating', 'time_avg_high_review', 'time_avg_low_review', 'effort_hours (per_week)', 'level', 'price', 
                  'enrollment_type', 'course_length', 'url', 'degree', 'closeness', 'betweenness', 
                 'eigenvector', 'transitivity', 'constraint']
    
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    dict_writer.writerows(courseList)
            